In [ ]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import csv

def lacunarity(image, box_sizes):
	lacunarity_values = []

	for box_size in box_sizes:
		mass_points = np.zeros(shape=(image.shape[0], image.shape[1]))

		for x in range(0, image.shape[0], box_size):
			for y in range(0, image.shape[1], box_size):
				box = image[x:x+box_size, y:y+box_size]
				mass_points[x // box_size, y // box_size] = np.sum(box >= 50)

		mean = np.mean(mass_points)
		std = np.std(mass_points)

		if mean == 0:
			lacunarity_value = 0
		else:
			lacunarity_value = (std / mean) ** 2

		lacunarity_values.append(lacunarity_value)

	return {
		"box_sizes": box_sizes,
		"lacunarity": lacunarity_values
	}

def plot_lacunarity(data, imageName, type, file):
	box_sizes = data["box_sizes"]
	lacunarity = data["lacunarity"]

	log_box_sizes = np.log(box_sizes)
	log_lacunarity = np.log(lacunarity)

	m, b = np.polyfit(log_box_sizes, log_lacunarity, 1)

	plt.figure(figsize=(10, 6))
	plt.plot(log_box_sizes, log_lacunarity, marker='o', label='Log(lacunaridade)')
	plt.plot(log_box_sizes, m * log_box_sizes + b, color='red', label='Regressão Linear')
	plt.xlabel('Log(tamanho da caixa)')
	plt.ylabel('Log(lacunaridade):')
	plt.title(f'Lacunaridade: {m}')
	plt.grid(True)
	plt.legend()
	plt.savefig(f'src/lac/{type}/{imageName}h.png')

	writer.writerow([imageName, m])

type = 'black'
imagens = ['00', '23', '48', '68', '75', '144']

with open(f'src/data/{type}/lac.csv', mode='w', newline='') as f:
	writer = csv.writer(f)

	for imageName in imagens:
		image = cv2.cvtColor(cv2.imread(f'src/{type}/{imageName}h.jpg'), cv2.COLOR_RGB2GRAY)
		box_sizes = np.logspace(start = 1, stop = int(np.log2(np.min(image.shape)) - 1), num = int(np.log2(np.min(image.shape)) - 1), base = 2).astype(int)

		data = lacunarity(image, box_sizes)
		plot_lacunarity(data, imageName, type, writer)